In [8]:
import pandas as pd
from mwviews.api import PageviewsClient
import numpy as np
def get_direct_link(shared_link):
    return 'https://drive.google.com/uc?export=download&id=' + shared_link.split('/')[-2]

link = "https://drive.google.com/file/d/1scglxYTopRbMg4X_RNoUT16chb8ulThG/view?usp=drive_link"

direct_link = get_direct_link(link)
players_df = pd.read_csv(direct_link)

end_date = '20211002'

In [9]:
def get_player_data(players):
    results = pd.DataFrame(columns=['Date', 'wiki', 'Player'])
    for player in players:
        
        if player == 'No Wikipedia page':
            nan_data = pd.DataFrame({
                'Date': pd.date_range(start='20150104', end=end_date),
                'wiki': [np.nan] * len(pd.date_range(start='20150104', end=end_date)),
                'Player': [player] * len(pd.date_range(start='20150104', end=end_date))
            })
            results = pd.concat([results, nan_data])
            continue 
            
        try:
            data = client.article_views('en.wikipedia', player,
                                        granularity='daily',
                                        start='20150104',
                                        end=end_date)
        except Exception as e:
            print(f"Error for player {player}: {str(e)}")
            continue

        data_df = pd.DataFrame.from_dict(data, orient='index')
        column = data_df.columns[0]
        data_df = data_df.reset_index()
        data_df.rename(columns={'index' : 'Date',
                                 column : 'wiki'}, inplace=True)
        data_df['Player'] = player

        date_mask = data_df['Date'].isin(results['Date'])
        player_mask = data_df['Player'].isin(results['Player'])
        dup_mask = date_mask & player_mask
        new_data = data_df[~dup_mask]
        if new_data.shape[0] == 0:
            continue

        new_data.fillna(0, inplace=True)
        new_data['year'] = new_data['Date'].dt.year
        new_data['month'] = new_data['Date'].dt.month
        new_data['day'] = new_data['Date'].dt.day
        results = pd.concat([results,new_data])

    return results

In [10]:
def combine_duplicate_players(df):
    grouped_players = df.groupby('wiki_title')['original_names'].apply(np.unique).reset_index()

    grouped_players['original_names'] = grouped_players['original_names'].apply(tuple)

    merged = df.merge(grouped_players, on='wiki_title', how='left')

    merged.drop(columns='original_names_x', inplace=True)
    merged.rename(columns={'original_names_y': 'original_names'}, inplace=True)

    merged.drop_duplicates(inplace=True)

    return merged


In [11]:
players = list(players_df['wiki_title'].dropna().unique())

In [13]:
# initialize client
client = PageviewsClient(user_agent="<jg19184@bristol.ac.uk> Buzz Factor Research")

# fetch wiki data
results = get_player_data(players)

ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/John_Casta%C3%B1eda/daily/2015010400/2021100200']
Error for player John Castañeda: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/John_Casta%C3%B1eda/daily/2015010400/2021100200']


Traceback (most recent call last):
  File "C:\Users\Archie Stevens\Documents\Anaconda\lib\site-packages\mwviews\api\pageviews.py", line 146, in article_views
    raise Exception(
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/John_Casta%C3%B1eda/daily/2015010400/2021100200']


ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Gloria_de_Paula/daily/2015010400/2021100200']
Error for player Gloria de Paula: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Gloria_de_Paula/daily/2015010400/2021100200']


Traceback (most recent call last):
  File "C:\Users\Archie Stevens\Documents\Anaconda\lib\site-packages\mwviews\api\pageviews.py", line 146, in article_views
    raise Exception(
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Gloria_de_Paula/daily/2015010400/2021100200']


ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Filip_Peji%C4%87/daily/2015010400/2021100200']
Error for player Filip Pejić: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Filip_Peji%C4%87/daily/2015010400/2021100200']


...

Traceback (most recent call last):
  File "C:\Users\Archie Stevens\Documents\Anaconda\lib\site-packages\mwviews\api\pageviews.py", line 146, in article_views
    raise Exception(
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Luis_Salda%C3%B1a/daily/2015010400/2021100200']


ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Aori_Qileng/daily/2015010400/2021100200']
Error for player Aori Qileng: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Aori_Qileng/daily/2015010400/2021100200']


Traceback (most recent call last):
  File "C:\Users\Archie Stevens\Documents\Anaconda\lib\site-packages\mwviews\api\pageviews.py", line 146, in article_views
    raise Exception(
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Aori_Qileng/daily/2015010400/2021100200']


ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Ignacio_Bahamondes/daily/2015010400/2021100200']
Error for player Ignacio Bahamondes: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Ignacio_Bahamondes/daily/2015010400/2021100200']


Traceback (most recent call last):
  File "C:\Users\Archie Stevens\Documents\Anaconda\lib\site-packages\mwviews\api\pageviews.py", line 146, in article_views
    raise Exception(
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Ignacio_Bahamondes/daily/2015010400/2021100200']


In [15]:
merged = players_df[['original_names', 'wiki_title']].merge\
  (results, left_on='wiki_title', right_on='Player',  how='left')
merged.drop(columns=['Player'], inplace=True)

print('There are this many null values:',merged.wiki.isnull().values.sum())
print('These are the null values:', merged.loc[merged.wiki.isnull()==True].wiki_title.values)


There are this many null values: 588970
These are the null values: ['No Wikipedia page' 'No Wikipedia page' 'No Wikipedia page' ...
 'No Wikipedia page' 'No Wikipedia page' 'No Wikipedia page']


In [20]:
merged = merged[merged.wiki.notnull()] 
nulls = merged.groupby('Date')['wiki'].sum() == 0  
null_dates = nulls[nulls == True].index.tolist()
print('Dates with no views:', nulls.loc[nulls == True].index) 
merged = merged[~merged.Date.isin(null_dates)]

Dates with no views: DatetimeIndex(['2015-01-04', '2015-01-05', '2015-01-06', '2015-01-07',
               '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11',
               '2015-01-12', '2015-01-13',
               ...
               '2015-06-21', '2015-06-22', '2015-06-23', '2015-06-24',
               '2015-06-25', '2015-06-26', '2015-06-27', '2015-06-28',
               '2015-06-29', '2015-06-30'],
              dtype='datetime64[ns]', name='Date', length=178, freq=None)


In [22]:
merged = combine_duplicate_players(merged)

In [24]:
merged.sort_values(['wiki_title', 'Date'], inplace=True)
merged.to_csv('ufc_wikipedia.csv', encoding='utf8', index=False)